In [73]:
# Underlying details extracted from nse

import pandas as pd
import requests
from bs4 import BeautifulSoup as bs4
import json

#******        Flags and Variables        ***
#_______________________________________________

load_from_pickle = True   # For underlying symbol list


#******   Error catch in list comprehension  ****
#________________________________________________

def catch(func, handle=lambda e : e, *args, **kwargs):
    '''List comprehension error catcher'''
    try:
        return func(*args, **kwargs)
    except Exception as e:
        pass
    
# get / make the symbols list
if load_from_pickle:
    symbols = pd.read_pickle('./zdata/nse_equity_options.pkl').Symbol.unique()
else:
    symbols = list(pd.read_csv('./zdata/nse_equity_symbols.csv'))

def get_underlying(symbol):
    
    '''Gets the underlying data
    Arg: (symbol as string)
    Returns: df as dataframe'''

    # URLs
    url_base1 = "https://www.nseindia.com/live_market/dynaContent/live_watch/"
    url = url_base + "get_quote/GetQuote.jsp?symbol="+symbol

    page = requests.get(url).text

    # soup out the json dict
    bs_nse = bs4(page, 'html.parser')
    json_nse = json.loads(bs_nse.find(id='responseDiv').text.strip())

    df = pd.DataFrame.from_dict(json_nse['data'][0], orient='index').T
    
    return df

nse_underlyings = [catch(lambda: get_underlying(symbol)) for symbol in symbols[:2]]

df_nse_und = pd.concat(nse_underlyings).set_index('symbol')

# clean up the commas and dashes
df_nse_und1 = df_nse_und.replace(',|-' , '', regex=True)

In [133]:
# convert column dates to datetime.date format

filtin = ['Date', 'dt']
filtout = ['isExDateFlag']
date_columns_mask = df_nse_und1.columns.str.contains('|'.join(filtin)) | (df_nse_und1.columns.str.contains('|'.join(filtout)))
date_column_list = df_nse_und1.loc[:, date_columns_mask].columns.tolist()
date_column_list = [item for item in date_column_list if item not in 'isExDateFlag']


In [134]:
date_column_list

['bcEndDate',
 'exDate',
 'bcStartDate',
 'cm_adj_high_dt',
 'secDate',
 'ndEndDate',
 'recordDate',
 'cm_adj_low_dt',
 'ndStartDate']

In [ ]:
df = df_nse_und1.loc[:, ['bcEndDate', 'exDate']].set_index(range(len(df_nse_und1)))
pd.to_datetime(df_nse_und1.loc[:, df], format='%d%B%y', errors='coerce')

In [100]:
df_nse_und1.loc[:, date_columns].columns

Index(['bcEndDate', 'exDate', 'bcStartDate', 'cm_adj_high_dt', 'secDate',
       'ndEndDate', 'recordDate', 'cm_adj_low_dt', 'ndStartDate',
       'isExDateFlag'],
      dtype='object')